In [1]:
%pip install llama-index-llms-litellm

%pip install llama-index


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install llama-index-graph-stores-neo4j


Note: you may need to restart the kernel to use updated packages.


# Knowledge Graph with Neo4j and LlamaIndex

This notebook creates a knowledge graph from documents using LlamaIndex and Neo4j.

In [3]:
import os
from llama_index.llms.litellm import LiteLLM
from llama_index.embeddings.openai import OpenAIEmbedding



In [4]:

import nest_asyncio
nest_asyncio.apply()

In [5]:
import dotenv
dotenv.load_dotenv()

True

In [6]:
llm = LiteLLM(model="gemini/gemini-2.0-flash-lite",api_key=os.getenv("GOOGLE_API_KEY"))
# llm = LiteLLM(model="ollama/qwen3:8b",api_key=os.getenv("GOOGLE_API_KEY"))
embedding = OpenAIEmbedding(model="text-embedding-ada-002", api_key=os.getenv("OPENAI_API_KEY"))

In [7]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from IPython.display import Markdown, display
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import StorageContext


In [8]:

# # For local Neo4j instance if you prefer that
username = "neo4j"
password = "password"  # default password for local Neo4j installation
url = "bolt://localhost:7687"  # standard localhost URL for Neo4j

In [9]:
documents= SimpleDirectoryReader("data").load_data()

In [10]:
graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database="neo4j",
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)


In [11]:

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    llm=llm,
    show_progress=True,
    embed_model=embedding,
)

f:\AI\KnowledgeGraph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Processing nodes: 100%|██████████| 37/37 [01:19<00:00,  2.15s/it]


# Querying and Retrieval


## Querying

In [12]:
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)

nodes = retriever.retrieve("ในสมการ (2.5) และ (2.6) ของ ACI มีเงื่อนไขการใช้สมการอย่างไร และตัวแปร h/r มีความสำคัญอย่างไร?")

for node in nodes:
    print(node.text)


The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
['ใช้', 'ศึกษาแรงกดอัด']
['ประกอบด้วย', 'อัตราส่วนความชะลูด']
['ประกอบด้วย', 'อัตราส่วนเหล็กเสริม']
['ประกอบด้วย', 'อัตราส่วนเหล็กเสริม', 'ทำให้', 'กำลังรับแรงดัดเพิ่มขึ้น']
['ไม่มี', 'พจน์']
['อยู่', 'รูป']


## Retrieval

In [13]:
query_engine = index.as_query_engine(include_text=True)

response = query_engine.query("ในสมการ (2.5) และ (2.6) ของ ACI มีเงื่อนไขการใช้สมการอย่างไร และตัวแปร h/r มีความสำคัญอย่างไร?")

print(str(response))

สมการ (2.5) และ (2.6) ของ ACI มีเงื่อนไขการใช้เช่น สำหรับ (2.5) เงื่อนไขคือ h/r ต้องน้อยกว่าหรือเท่ากับ 99 และสำหรับ (2.6) เงื่อนไขคือ h/r ต้องมากกว่า 99 โดยตัวแปร h/r มีความสำคัญเนื่องจากมีผลต่อการคำนวณแรงบรรทุกในแนวดิ่วที่ยอมให้ที่กระทำบนผนังในสมการทั้งสองในทางทฤษฎีและการออกแบบ.
